In [1]:
%run StdPackages.ipynb

No clean-up of work-folder


*Load test database:*

In [2]:
db = GpyDB(db = os.path.join(d['data'], 'AbatementData.gdx'))

# GmsPy.Compile

The class is used to split variables into exogenous and endogenous groups of variables relying on symbols from a database (```GpyDB```). The ```Compile``` class is based on the ```Group``` - we will start by defining this.

### ```GmsPy.Group```

In [3]:
os.chdir(d['py'])
from GmsPy import Group
os.chdir(d['curr'])

We can define groups of variables using four attributes:
* ```self.v```: List of tuples. Each tuple is length 2, with first element indicating the variable, the second indicating the condition on that variable.
* ```self.g```: Ordered set. Each element is a string referencing other groups to be included.
* ```self.neg_v```: List of tuples akin to ```self.v```. Each element is subtracted from the group.
* ```sef.neg_g```: Ordered set. Each element is a string referencing another group that is subtracted.

When specifying variables, the conditions should conform to the form used in ```pyDatabases.gpyDB_wheels.adj``` and ```GmsWrite.writeGpy```.

*NB: As groups can depend on other groups, it can be important to process groups in the right order.*

In [4]:
groups = {}

#### Group examples:

*1. Add a group specified as a list of tuples:*

Specify a group called ```g1``` by specifying a list of tuples to include:

In [5]:
g1 = Group('g1', 
           v = [('theta',db['V01_T']), # V01_T is a dummy, so we only use a part of the symbol 'theta'
                ('mu', ('and', [db['V01_inp2T'], ('not', db['V01_dur'])])), # use symbols from dummy V01_inp2T, but not in V01_dur
                ('sigma', db['V01_T2ESNorm'])])

When groups are *compiled*, we construct two main dicts ```self.out, self.out_neg```:

In [6]:
groups[g1.name] = g1.compile(groups)

The ```self.out``` part contains elements that are added to the group, ```self.out_neg``` are elements to extract from a group definition. We can use this to fix/unfix variables (see below).

*2. Add a group specified by referencing a group and variables:*

Specify list of variables + reference group 'g1'. This adds a variable 'mu' and the conditional of the mapping 'V01_NT_inp' that has been aliased with 'n','nn'. A preview of what will eventually be written:

In [7]:
c = {'s': db['V01_NT_inp'], 'alias': {'n':'nn'}}
GmsWrite.writeGpy(**c)

'V01_NT_inp[s,nn]'

In [8]:
g2 = Group('g2', v = [('mu',c)], g=['g1'])

Compile:

In [9]:
groups[g2.name] = g2.compile(groups)
groups[g2.name].out

{'mu': [{'s': <pyDatabases.gpyDB._database.gpy at 0x1cedcbba730>,
   'alias': {'n': 'nn'}},
  ('and',
    ('not', <pyDatabases.gpyDB._database.gpy at 0x1cecb1666d0>)])],
 'theta': [<pyDatabases.gpyDB._database.gpy at 0x1cecb23f880>],
 'sigma': [<pyDatabases.gpyDB._database.gpy at 0x1cecb23ff40>]}

#### Subtract a group

Define a group as the group 'g2', but extract 'g1':

In [10]:
g3 = Group('g3', g = ['g2'], neg_g = ['g1'])

Compile:

In [11]:
groups[g3.name] = g3.compile(groups)
groups[g3.name].out

{'mu': [('or',
   [{'s': <pyDatabases.gpyDB._database.gpy at 0x1cedcbba730>,
     'alias': {'n': 'nn'}},
    ('and',
      ('not', <pyDatabases.gpyDB._database.gpy at 0x1cecb1666d0>)])])]}

In [12]:
groups[g3.name].out_neg

{'mu': [('and',
    ('not', <pyDatabases.gpyDB._database.gpy at 0x1cecb1666d0>)])]}

#### Subtract a group and a variable:

In [13]:
g4 = Group('g4', g = ['g2'], neg_g = ['g1'], neg_v = [('mu', db['V01_NT_inp'])])

Compile:

In [14]:
groups[g4.name] = g4.compile(groups)
groups[g4.name].out, g4.out_neg

({'mu': [('or',
    [{'s': <pyDatabases.gpyDB._database.gpy at 0x1cedcbba730>,
      'alias': {'n': 'nn'}},
     ('and',
       ('not', <pyDatabases.gpyDB._database.gpy at 0x1cecb1666d0>)])])]},
 {'mu': [<pyDatabases.gpyDB._database.gpy at 0x1cedcbba730>,
   ('and',
     ('not', <pyDatabases.gpyDB._database.gpy at 0x1cecb1666d0>)])]})

#### Collect conditions from a group:

The ```self.compile``` creates a dictionary of conditions to apply (```self.out```) and conditions to negate and apply (```self.out_neg```). The two are combined into one dictionary of nested conditions using the ```self.conditions``` property. These can be processed using e.g. the ```writeGpy``` method:

In [15]:
for g in groups.values():
    print(f"Text for group {g.name}:")
    for k,v in g.conditions.items():
        print(f"\t{GmsWrite.writeGpy(db[k],c=v)}")

Text for group g1:
	theta[s,n]$(V01_T[s,n])
	mu[s,n,nn]$((V01_inp2T[s,n,nn] and ( not (V01_dur[s,n]))))
	sigma[s,n]$(V01_T2ESNorm[s,n,nn])
Text for group g2:
	mu[s,n,nn]$((V01_NT_inp[s,nn] or (V01_inp2T[s,n,nn] and ( not (V01_dur[s,n])))))
	theta[s,n]$(V01_T[s,n])
	sigma[s,n]$(V01_T2ESNorm[s,n,nn])
Text for group g3:
	mu[s,n,nn]$(((V01_NT_inp[s,nn] or (V01_inp2T[s,n,nn] and ( not (V01_dur[s,n])))) and ( not ((V01_inp2T[s,n,nn] and ( not (V01_dur[s,n])))))))
Text for group g4:
	mu[s,n,nn]$(((V01_NT_inp[s,nn] or (V01_inp2T[s,n,nn] and ( not (V01_dur[s,n])))) and ( not ((V01_NT_inp[s,n] or (V01_inp2T[s,n,nn] and ( not (V01_dur[s,n]))))))))


### ```GmsPy.Compile```

The class is a small wrapper around a dictionary of ```GmsPy.Groups```. The class has a few methods that helps us write some standard codes of GAMS blocks, but also to define metablocks in a fast way. Let's use the ```groups``` dictionary from before as the collection of groups:

In [16]:
c = GmsPy.Compile(groups = groups)

In the previous section, we have compiled each of the groups separately. If we hadn't done that, the ```self.run``` method does this for us:

In [17]:
c.run()

We can now establish meta-groups, by specifying a list of groups names:

In [18]:
metagroup = c.metaGroup(db, gs = ['g1','g2'])

If we do not pass any argument ```gs```, we default to ```gs='all'``` in which case all groups are collected in one:

In [19]:
metagroup_all = c.metaGroup(db)

Finally, we can use this to write syntax used for fixing/unfixing variables. Say, for instance, that two of our groups are meant to be endogenous, and two of them are supposed to be exogenous. We get the relevant GAMS text needed to do this by calling:

*Fix variables at levels:*

In [20]:
print(c.fixGroupsText(db, ['g1','g2']))

theta.fx[s,n]$((V01_T[s,n] or V01_T[s,n])) = theta.l[s,n];
mu.fx[s,n,nn]$(((V01_inp2T[s,n,nn] and ( not (V01_dur[s,n]))) or (V01_NT_inp[s,nn] or (V01_inp2T[s,n,nn] and ( not (V01_dur[s,n])))))) = mu.l[s,n,nn];
sigma.fx[s,n]$((V01_T2ESNorm[s,n,nn] or V01_T2ESNorm[s,n,nn])) = sigma.l[s,n];


*Unfix variables:*

In [21]:
print(c.unfixGroupsText(db, ['g3','g4']))

mu.lo[s,n,nn]$((((V01_NT_inp[s,nn] or (V01_inp2T[s,n,nn] and ( not (V01_dur[s,n])))) and ( not ((V01_inp2T[s,n,nn] and ( not (V01_dur[s,n])))))) or ((V01_NT_inp[s,nn] or (V01_inp2T[s,n,nn] and ( not (V01_dur[s,n])))) and ( not ((V01_NT_inp[s,n] or (V01_inp2T[s,n,nn] and ( not (V01_dur[s,n]))))))))) = -inf;
mu.up[s,n,nn]$((((V01_NT_inp[s,nn] or (V01_inp2T[s,n,nn] and ( not (V01_dur[s,n])))) and ( not ((V01_inp2T[s,n,nn] and ( not (V01_dur[s,n])))))) or ((V01_NT_inp[s,nn] or (V01_inp2T[s,n,nn] and ( not (V01_dur[s,n])))) and ( not ((V01_NT_inp[s,n] or (V01_inp2T[s,n,nn] and ( not (V01_dur[s,n]))))))))) = inf;


### EXPORT ABATEMENT EXAMPLE TO PICKLE:

In [22]:
g1 = GmsPy.Group('G_V01_NT_endo_always', v = [('pS', db['V01_NT_out']),
                                        ('pD', db['V01_NT_int']),
                                        ('qD', db['V01_NT_x'])])
g2 = GmsPy.Group('G_V01_NT_endo_base', v = [('qD', db['V01_NT_int']),
                                      ('qD', ('and', [db['V01_NT_inp'], ('not', db['V01_NT_x'])]))])
g3 = GmsPy.Group('G_V01_NT_exo_always', v = [('sigma', db['V01_NT_int']),
                                       ('sigma', db['V01_NT_out']),
                                       ('pD', db['V01_inp']),
                                       ('qS', db['V01_NT_out'])])
g4 = GmsPy.Group('G_V01_NT_exo_base', v = [('mu', db['V01_NT_map'])])
g5 = GmsPy.Group('G_V01_T_endo_always', v = [('lambda', db['V01_ES']),
                                       ('pD', db['V01_ES'])])
g6 = GmsPy.Group('G_V01_T_endo_base', v = [('qD', db['V01_T']),
                                     ('pD', db['V01_T'])])
g7 = GmsPy.Group('G_V01_T_exo_always', v = [('theta', db['V01_T']),
                                      ('mu', ('and', [db['V01_inp2T'], ('not', db['V01_dur'])])),
                                      ('sigma', db['V01_ES']),
                                      ('mu', db['V01_T2ESNorm'])])
g8 = GmsPy.Group('G_V01_T_exo_base', v = [('mu', ('and', [db['V01_T2ES'], ('not', db['V01_T2ESNorm'])])),
                                    ('mu', ('and', [db['V01_inp2T'], db['V01_dur']]))])
g9 = GmsPy.Group('G_V01_ACC_endo_base', v = [('qD', db['V01_inp'])])

Initialize settings with groups:

In [23]:
c = GmsPy.Compile(groups = {g.name: g for g in (g1,g2,g3,g4,g5,g6,g7,g8,g9)})
with open(os.path.join(d['data'], 'AbatementGroups'), "wb") as file:
    pickle.dump(c,file)